# PIL (글자만)

In [ ]:
"""
Data aumentation
1. 여러 폰트별 글자 생성 (현재 47개 확보) -> 구현완료
2. 글씨크기 조절 -> 구현완료
3. 글씨색깔 조절(?)
4. 배경색깔 조절(?)
5. 이미지 자체의 크기 조절
6. 짧은글의 개수 -> 무조건 3개는 아니야


추가로 가능한지 찾아보자
1. 글자 distortion --> 전단지 글씨에 이런 것이 필요할까?
2. 글자 회전 --> 전단지 글씨에 이런 것이 필요할까?
3. 글자 blur --> 이것은 필요할듯

"""

In [14]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageChops
import pandas as pd
import numpy as np
import os
import time
import random
import warnings

warnings.filterwarnings(action='ignore')

# 폴더 생성
if 'data' not in os.listdir():
    os.mkdir('data')

if 'train' not in os.listdir('data/'):
    os.mkdir('data/train/')
    
if 'test' not in os.listdir('data/'):
    os.mkdir('data/test/')
    
if 'train_label' not in os.listdir('data/'):
    os.mkdir('data/train_label/')
    
if 'test_label' not in os.listdir('data/'):
    os.mkdir('data/test_label/')
    
# 공백제거 함수
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -25)
    bbox = diff.getbbox()
    bbox = (bbox[0]-2, bbox[1]-2, bbox[2]+2, bbox[3]+2)
    
    if bbox:
        return im.crop(bbox)

In [15]:
# 짧은글(뉴스의 제목)데이터 가져오기
text_samples = pd.read_excel('sample_data.xlsx')['제목']
eng_texts = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
text_labels = ' +\'"~@#$%^&_=()*,-./0123456789:<>[]!?abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰갱갸갹갼걀걋걍걔걘걜거걱건걷걸걺검겁것겄겅겆겉겊겋게겐겔겜겝겟겠겡겨격겪견겯결겸겹겻겼경곁계곈곌곕곗고곡곤곧골곪곬곯곰곱곳공곶과곽관괄괆괌괍괏광괘괜괠괩괬괭괴괵괸괼굄굅굇굉교굔굘굡굣구국군굳굴굵굶굻굼굽굿궁궂궈궉권궐궜궝궤궷귀귁귄귈귐귑귓규균귤그극근귿글긁금급긋긍긔기긱긴긷길긺김깁깃깅깆깊까깍깎깐깔깖깜깝깟깠깡깥깨깩깬깰깸깹깻깼깽꺄꺅꺌꺼꺽꺾껀껄껌껍껏껐껑께껙껜껨껫껭껴껸껼꼇꼈꼍꼐꼬꼭꼰꼲꼴꼼꼽꼿꽁꽂꽃꽈꽉꽐꽜꽝꽤꽥꽹꾀꾄꾈꾐꾑꾕꾜꾸꾹꾼꿀꿇꿈꿉꿋꿍꿎꿔꿜꿨꿩꿰꿱꿴꿸뀀뀁뀄뀌뀐뀔뀜뀝뀨끄끅끈끊끌끎끓끔끕끗끙끝끼끽낀낄낌낍낏낑나낙낚난낟날낡낢남납낫났낭낮낯낱낳내낵낸낼냄냅냇냈냉냐냑냔냘냠냥너넉넋넌널넒넓넘넙넛넜넝넣네넥넨넬넴넵넷넸넹녀녁년녈념녑녔녕녘녜녠노녹논놀놂놈놉놋농높놓놔놘놜놨뇌뇐뇔뇜뇝뇟뇨뇩뇬뇰뇹뇻뇽누눅눈눋눌눔눕눗눙눠눴눼뉘뉜뉠뉨뉩뉴뉵뉼늄늅늉느늑는늘늙늚늠늡늣능늦늪늬늰늴니닉닌닐닒님닙닛닝닢다닥닦단닫달닭닮닯닳담답닷닸당닺닻닿대댁댄댈댐댑댓댔댕댜더덕덖던덛덜덞덟덤덥덧덩덫덮데덱덴델뎀뎁뎃뎄뎅뎌뎐뎔뎠뎡뎨뎬도독돈돋돌돎돐돔돕돗동돛돝돠돤돨돼됐되된될됨됩됫됴두둑둔둘둠둡둣둥둬뒀뒈뒝뒤뒨뒬뒵뒷뒹듀듄듈듐듕드득든듣들듦듬듭듯등듸디딕딘딛딜딤딥딧딨딩딪따딱딴딸땀땁땃땄땅땋때땍땐땔땜땝땟땠땡떠떡떤떨떪떫떰떱떳떴떵떻떼떽뗀뗄뗌뗍뗏뗐뗑뗘뗬또똑똔똘똥똬똴뙈뙤뙨뚜뚝뚠뚤뚫뚬뚱뛔뛰뛴뛸뜀뜁뜅뜨뜩뜬뜯뜰뜸뜹뜻띄띈띌띔띕띠띤띨띰띱띳띵라락란랄람랍랏랐랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝려력련렬렴렵렷렸령례롄롑롓로록론롤롬롭롯롱롸롼뢍뢨뢰뢴뢸룀룁룃룅료룐룔룝룟룡루룩룬룰룸룹룻룽뤄뤘뤠뤼뤽륀륄륌륏륑류륙륜률륨륩륫륭르륵른를름릅릇릉릊릍릎리릭린릴림립릿링마막만많맏말맑맒맘맙맛망맞맡맣매맥맨맬맴맵맷맸맹맺먀먁먈먕머먹먼멀멂멈멉멋멍멎멓메멕멘멜멤멥멧멨멩며멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏뭐뭔뭘뭡뭣뭬뮈뮌뮐뮤뮨뮬뮴뮷므믄믈믐믓미믹민믿밀밂밈밉밋밌밍및밑바박밖밗반받발밝밞밟밤밥밧방밭배백밴밸뱀뱁뱃뱄뱅뱉뱌뱍뱐뱝버벅번벋벌벎범법벗벙벚베벡벤벧벨벰벱벳벴벵벼벽변별볍볏볐병볕볘볜보복볶본볼봄봅봇봉봐봔봤봬뵀뵈뵉뵌뵐뵘뵙뵤뵨부북분붇불붉붊붐붑붓붕붙붚붜붤붰붸뷔뷕뷘뷜뷩뷰뷴뷸븀븃븅브븍븐블븜븝븟비빅빈빌빎빔빕빗빙빚빛빠빡빤빨빪빰빱빳빴빵빻빼빽뺀뺄뺌뺍뺏뺐뺑뺘뺙뺨뻐뻑뻔뻗뻘뻠뻣뻤뻥뻬뼁뼈뼉뼘뼙뼛뼜뼝뽀뽁뽄뽈뽐뽑뽕뾔뾰뿅뿌뿍뿐뿔뿜뿟뿡쀼쁑쁘쁜쁠쁨쁩삐삑삔삘삠삡삣삥사삭삯산삳살삵삶삼삽삿샀상샅새색샌샐샘샙샛샜생샤샥샨샬샴샵샷샹섀섄섈섐섕서석섞섟선섣설섦섧섬섭섯섰성섶세섹센셀셈셉셋셌셍셔셕션셜셤셥셧셨셩셰셴셸솅소속솎손솔솖솜솝솟송솥솨솩솬솰솽쇄쇈쇌쇔쇗쇘쇠쇤쇨쇰쇱쇳쇼쇽숀숄숌숍숏숑수숙순숟술숨숩숫숭숯숱숲숴쉈쉐쉑쉔쉘쉠쉥쉬쉭쉰쉴쉼쉽쉿슁슈슉슐슘슛슝스슥슨슬슭슴습슷승시식신싣실싫심십싯싱싶싸싹싻싼쌀쌈쌉쌌쌍쌓쌔쌕쌘쌜쌤쌥쌨쌩썅써썩썬썰썲썸썹썼썽쎄쎈쎌쏀쏘쏙쏜쏟쏠쏢쏨쏩쏭쏴쏵쏸쐈쐐쐤쐬쐰쐴쐼쐽쑈쑤쑥쑨쑬쑴쑵쑹쒀쒔쒜쒸쒼쓩쓰쓱쓴쓸쓺쓿씀씁씌씐씔씜씨씩씬씰씸씹씻씽아악안앉않알앍앎앓암압앗았앙앝앞애액앤앨앰앱앳앴앵야약얀얄얇얌얍얏양얕얗얘얜얠얩어억언얹얻얼얽얾엄업없엇었엉엊엌엎에엑엔엘엠엡엣엥여역엮연열엶엷염엽엾엿였영옅옆옇예옌옐옘옙옛옜오옥온올옭옮옰옳옴옵옷옹옻와왁완왈왐왑왓왔왕왜왝왠왬왯왱외왹왼욀욈욉욋욍요욕욘욜욤욥욧용우욱운울욹욺움웁웃웅워웍원월웜웝웠웡웨웩웬웰웸웹웽위윅윈윌윔윕윗윙유육윤율윰윱윳융윷으윽은을읊음읍읏응읒읓읔읕읖읗의읜읠읨읫이익인일읽읾잃임입잇있잉잊잎자작잔잖잗잘잚잠잡잣잤장잦재잭잰잴잼잽잿쟀쟁쟈쟉쟌쟎쟐쟘쟝쟤쟨쟬저적전절젊점접젓정젖제젝젠젤젬젭젯젱져젼졀졈졉졌졍졔조족존졸졺좀좁좃종좆좇좋좌좍좔좝좟좡좨좼좽죄죈죌죔죕죗죙죠죡죤죵주죽준줄줅줆줌줍줏중줘줬줴쥐쥑쥔쥘쥠쥡쥣쥬쥰쥴쥼즈즉즌즐즘즙즛증지직진짇질짊짐집짓징짖짙짚짜짝짠짢짤짧짬짭짯짰짱째짹짼쨀쨈쨉쨋쨌쨍쨔쨘쨩쩌쩍쩐쩔쩜쩝쩟쩠쩡쩨쩽쪄쪘쪼쪽쫀쫄쫌쫍쫏쫑쫓쫘쫙쫠쫬쫴쬈쬐쬔쬘쬠쬡쭁쭈쭉쭌쭐쭘쭙쭝쭤쭸쭹쮜쮸쯔쯤쯧쯩찌찍찐찔찜찝찡찢찧차착찬찮찰참찹찻찼창찾채책챈챌챔챕챗챘챙챠챤챦챨챰챵처척천철첨첩첫첬청체첵첸첼쳄쳅쳇쳉쳐쳔쳤쳬쳰촁초촉촌촐촘촙촛총촤촨촬촹최쵠쵤쵬쵭쵯쵱쵸춈추축춘출춤춥춧충춰췄췌췐취췬췰췸췹췻췽츄츈츌츔츙츠측츤츨츰츱츳층치칙친칟칠칡침칩칫칭카칵칸칼캄캅캇캉캐캑캔캘캠캡캣캤캥캬캭컁커컥컨컫컬컴컵컷컸컹케켁켄켈켐켑켓켕켜켠켤켬켭켯켰켱켸코콕콘콜콤콥콧콩콰콱콴콸쾀쾅쾌쾡쾨쾰쿄쿠쿡쿤쿨쿰쿱쿳쿵쿼퀀퀄퀑퀘퀭퀴퀵퀸퀼큄큅큇큉큐큔큘큠크큭큰클큼큽킁키킥킨킬킴킵킷킹타탁탄탈탉탐탑탓탔탕태택탠탤탬탭탯탰탱탸턍터턱턴털턺텀텁텃텄텅테텍텐텔템텝텟텡텨텬텼톄톈토톡톤톨톰톱톳통톺톼퇀퇘퇴퇸툇툉툐투툭툰툴툼툽툿퉁퉈퉜퉤튀튁튄튈튐튑튕튜튠튤튬튱트특튼튿틀틂틈틉틋틔틘틜틤틥티틱틴틸팀팁팃팅파팍팎판팔팖팜팝팟팠팡팥패팩팬팰팸팹팻팼팽퍄퍅퍼퍽펀펄펌펍펏펐펑페펙펜펠펨펩펫펭펴편펼폄폅폈평폐폘폡폣포폭폰폴폼폽폿퐁퐈퐝푀푄표푠푤푭푯푸푹푼푿풀풂품풉풋풍풔풩퓌퓐퓔퓜퓟퓨퓬퓰퓸퓻퓽프픈플픔픕픗피픽핀필핌핍핏핑하학한할핥함합핫항해핵핸핼햄햅햇했행햐향허헉헌헐헒험헙헛헝헤헥헨헬헴헵헷헹혀혁현혈혐협혓혔형혜혠혤혭호혹혼홀홅홈홉홋홍홑화확환활홧황홰홱홴횃횅회획횐횔횝횟횡효횬횰횹횻후훅훈훌훑훔훗훙훠훤훨훰훵훼훽휀휄휑휘휙휜휠휨휩휫휭휴휵휸휼흄흇흉흐흑흔흖흗흘흙흠흡흣흥흩희흰흴흼흽힁히힉힌힐힘힙힛힝'

for idx in range(len(text_samples)):
    if False in [text in text_labels for text in text_samples[idx]]:
        text_samples.drop(idx, inplace=True)
        
text_samples = text_samples.tolist()
print(f'짧은글의 개수: {len(text_samples)}')

짧은글의 개수: 13193


In [16]:
# 폰트데이터 경로 불러오기
font_paths = [f'fonts/ko/{path}' for path in os.listdir('fonts/ko')]
print(f'폰트개수: {len(font_paths)}')

폰트개수: 8


In [17]:
# 파라미터 정의
FONT_SIZES = [12, 14, 16, 18, 20]

BACKGROUND_SIZE = (1000, 100) 
BACKGROUND_COLORS = ['WHITE', 'BLACK', 'RED', 'YELLOW'] # 이미지 배경색깔

TEXT_COLORS = ['black', 'white', 'red', 'yellow'] # 글씨 색깔
TEXT_ALIGN = 'center' # 글씨 정렬방법

In [126]:
def textimagegenerator(NUM_IMAGES, save_image_path, save_label_path):

    # 초깃값 정의
    cnt = 1
    t1 = time.time()
    df = pd.DataFrame()
    
    while True:

        # 배경색 및 글자색 선정
        if np.random.rand(1)[0] >= 0.2:
            BACKGROUND_COLOR = 'WHITE'
            TEXT_COLOR = 'black'

        else:
            BACKGROUND_COLOR = BACKGROUND_COLORS[np.random.randint(len(BACKGROUND_COLORS))]
            TEXT_COLOR = TEXT_COLORS[np.random.randint(len(TEXT_COLORS))]

        # 색이 같으면 생성하지 않음
        if BACKGROUND_COLOR == TEXT_COLOR.upper():
            continue

        # Text의 크기 및 폰트 설정    
        font_size = FONT_SIZES[np.random.randint(len(FONT_SIZES))]
        font_path = font_paths[np.random.randint(len(font_paths))]
        FONT = ImageFont.truetype(font_path, font_size) # 폰트정보: (폰트명, 글씨크기)
        
        if np.random.rand(1)[0] >= 0.1:
            TEXT = ' '.join(text_samples[np.random.randint(len(text_samples))].split(' ')[:3])
        else:
            TEXT = f'{"".join([random.choice(eng_texts) for i in range(1, np.random.randint(2, 10))])} {"".join([random.choice(eng_texts) for i in range(1, np.random.randint(2, 10))])} {"".join([random.choice(eng_texts) for i in range(1, np.random.randint(2, 10))])}'         
        
        # 글자가 들어간 그림 생성
        image = Image.new(mode = "RGB", # (RGB여부, 이미지크기, 배경색깔)
                          size = BACKGROUND_SIZE, 
                          color = BACKGROUND_COLOR) 

        draw = ImageDraw.Draw(image)
        w, _ = draw.textsize(TEXT, font=FONT)
        draw.text((5, 5), # (시작위치, 적고자 하는 TEXT, 폰트정보, 글씨색, 정렬위치)
                  TEXT, 
                  font = FONT, 
                  fill = TEXT_COLOR, 
                  align = TEXT_ALIGN) 

        # 이미지 blur효과 (넣거나, 말거나)
        if np.random.rand(1)[0] >= 0.2:
            image = trim(image)

        else:
            image = image.filter(ImageFilter.GaussianBlur(np.random.randint(2)))
            image = trim(image)

        # 이미지 및 라벨 저장
        image_path = f'{save_image_path}/sample{cnt}.png'
        image.save(image_path)
        tmp = pd.DataFrame({'image_path':[image_path],
                           'label':[TEXT]})
        
        df = pd.concat([df, tmp]).reset_index(drop=True)
        

        # 로그출력
        if cnt % 100 == 0:
            print(f'{cnt}개 이미지 생성 완료!! == 경과시간: {round(time.time() - t1)}초')

        # 조기종료조건
        if cnt == NUM_IMAGES:
            print(f'조기종료 == 경과시간: {round(time.time() - t1)}초')
            break

        # 개수 추가
        cnt += 1
    
    df.to_csv(f'{save_label_path}/label.csv', index=False, encoding='utf-8-sig')
    print('finish!!')

In [127]:
textimagegenerator(NUM_IMAGES = 150000,
                   save_image_path = 'data/train',
                   save_label_path = 'data/train_label')

100개 이미지 생성 완료!! == 경과시간: 1초
200개 이미지 생성 완료!! == 경과시간: 2초
300개 이미지 생성 완료!! == 경과시간: 2초
400개 이미지 생성 완료!! == 경과시간: 3초
500개 이미지 생성 완료!! == 경과시간: 4초
600개 이미지 생성 완료!! == 경과시간: 5초
700개 이미지 생성 완료!! == 경과시간: 5초
800개 이미지 생성 완료!! == 경과시간: 6초
900개 이미지 생성 완료!! == 경과시간: 7초
1000개 이미지 생성 완료!! == 경과시간: 8초
1100개 이미지 생성 완료!! == 경과시간: 9초
1200개 이미지 생성 완료!! == 경과시간: 9초
1300개 이미지 생성 완료!! == 경과시간: 10초
1400개 이미지 생성 완료!! == 경과시간: 11초
1500개 이미지 생성 완료!! == 경과시간: 12초
1600개 이미지 생성 완료!! == 경과시간: 13초
1700개 이미지 생성 완료!! == 경과시간: 13초
1800개 이미지 생성 완료!! == 경과시간: 14초
1900개 이미지 생성 완료!! == 경과시간: 15초
2000개 이미지 생성 완료!! == 경과시간: 16초
2100개 이미지 생성 완료!! == 경과시간: 17초
2200개 이미지 생성 완료!! == 경과시간: 18초
2300개 이미지 생성 완료!! == 경과시간: 18초
2400개 이미지 생성 완료!! == 경과시간: 19초
2500개 이미지 생성 완료!! == 경과시간: 20초
2600개 이미지 생성 완료!! == 경과시간: 21초
2700개 이미지 생성 완료!! == 경과시간: 22초
2800개 이미지 생성 완료!! == 경과시간: 23초
2900개 이미지 생성 완료!! == 경과시간: 24초
3000개 이미지 생성 완료!! == 경과시간: 24초
3100개 이미지 생성 완료!! == 경과시간: 25초
3200개 이미지 생성 완료!! == 경과시간: 26초
3300개 이미지 생성 완료!! == 경과시간: 27

25700개 이미지 생성 완료!! == 경과시간: 272초
25800개 이미지 생성 완료!! == 경과시간: 274초
25900개 이미지 생성 완료!! == 경과시간: 275초
26000개 이미지 생성 완료!! == 경과시간: 276초
26100개 이미지 생성 완료!! == 경과시간: 277초
26200개 이미지 생성 완료!! == 경과시간: 278초
26300개 이미지 생성 완료!! == 경과시간: 280초
26400개 이미지 생성 완료!! == 경과시간: 281초
26500개 이미지 생성 완료!! == 경과시간: 282초
26600개 이미지 생성 완료!! == 경과시간: 283초
26700개 이미지 생성 완료!! == 경과시간: 284초
26800개 이미지 생성 완료!! == 경과시간: 286초
26900개 이미지 생성 완료!! == 경과시간: 287초
27000개 이미지 생성 완료!! == 경과시간: 288초
27100개 이미지 생성 완료!! == 경과시간: 289초
27200개 이미지 생성 완료!! == 경과시간: 290초
27300개 이미지 생성 완료!! == 경과시간: 291초
27400개 이미지 생성 완료!! == 경과시간: 293초
27500개 이미지 생성 완료!! == 경과시간: 294초
27600개 이미지 생성 완료!! == 경과시간: 295초
27700개 이미지 생성 완료!! == 경과시간: 296초
27800개 이미지 생성 완료!! == 경과시간: 297초
27900개 이미지 생성 완료!! == 경과시간: 298초
28000개 이미지 생성 완료!! == 경과시간: 300초
28100개 이미지 생성 완료!! == 경과시간: 301초
28200개 이미지 생성 완료!! == 경과시간: 302초
28300개 이미지 생성 완료!! == 경과시간: 303초
28400개 이미지 생성 완료!! == 경과시간: 304초
28500개 이미지 생성 완료!! == 경과시간: 305초
28600개 이미지 생성 완료!! == 경과시간: 306초
28700개 이미지

50600개 이미지 생성 완료!! == 경과시간: 621초
50700개 이미지 생성 완료!! == 경과시간: 623초
50800개 이미지 생성 완료!! == 경과시간: 624초
50900개 이미지 생성 완료!! == 경과시간: 626초
51000개 이미지 생성 완료!! == 경과시간: 628초
51100개 이미지 생성 완료!! == 경과시간: 629초
51200개 이미지 생성 완료!! == 경과시간: 631초
51300개 이미지 생성 완료!! == 경과시간: 632초
51400개 이미지 생성 완료!! == 경과시간: 634초
51500개 이미지 생성 완료!! == 경과시간: 636초
51600개 이미지 생성 완료!! == 경과시간: 637초
51700개 이미지 생성 완료!! == 경과시간: 639초
51800개 이미지 생성 완료!! == 경과시간: 640초
51900개 이미지 생성 완료!! == 경과시간: 642초
52000개 이미지 생성 완료!! == 경과시간: 644초
52100개 이미지 생성 완료!! == 경과시간: 645초
52200개 이미지 생성 완료!! == 경과시간: 647초
52300개 이미지 생성 완료!! == 경과시간: 648초
52400개 이미지 생성 완료!! == 경과시간: 650초
52500개 이미지 생성 완료!! == 경과시간: 651초
52600개 이미지 생성 완료!! == 경과시간: 653초
52700개 이미지 생성 완료!! == 경과시간: 654초
52800개 이미지 생성 완료!! == 경과시간: 656초
52900개 이미지 생성 완료!! == 경과시간: 658초
53000개 이미지 생성 완료!! == 경과시간: 659초
53100개 이미지 생성 완료!! == 경과시간: 661초
53200개 이미지 생성 완료!! == 경과시간: 662초
53300개 이미지 생성 완료!! == 경과시간: 664초
53400개 이미지 생성 완료!! == 경과시간: 666초
53500개 이미지 생성 완료!! == 경과시간: 667초
53600개 이미지

75400개 이미지 생성 완료!! == 경과시간: 1085초
75500개 이미지 생성 완료!! == 경과시간: 1087초
75600개 이미지 생성 완료!! == 경과시간: 1089초
75700개 이미지 생성 완료!! == 경과시간: 1091초
75800개 이미지 생성 완료!! == 경과시간: 1093초
75900개 이미지 생성 완료!! == 경과시간: 1095초
76000개 이미지 생성 완료!! == 경과시간: 1098초
76100개 이미지 생성 완료!! == 경과시간: 1100초
76200개 이미지 생성 완료!! == 경과시간: 1102초
76300개 이미지 생성 완료!! == 경과시간: 1104초
76400개 이미지 생성 완료!! == 경과시간: 1106초
76500개 이미지 생성 완료!! == 경과시간: 1108초
76600개 이미지 생성 완료!! == 경과시간: 1110초
76700개 이미지 생성 완료!! == 경과시간: 1113초
76800개 이미지 생성 완료!! == 경과시간: 1115초
76900개 이미지 생성 완료!! == 경과시간: 1117초
77000개 이미지 생성 완료!! == 경과시간: 1119초
77100개 이미지 생성 완료!! == 경과시간: 1121초
77200개 이미지 생성 완료!! == 경과시간: 1123초
77300개 이미지 생성 완료!! == 경과시간: 1125초
77400개 이미지 생성 완료!! == 경과시간: 1127초
77500개 이미지 생성 완료!! == 경과시간: 1130초
77600개 이미지 생성 완료!! == 경과시간: 1132초
77700개 이미지 생성 완료!! == 경과시간: 1134초
77800개 이미지 생성 완료!! == 경과시간: 1136초
77900개 이미지 생성 완료!! == 경과시간: 1138초
78000개 이미지 생성 완료!! == 경과시간: 1140초
78100개 이미지 생성 완료!! == 경과시간: 1142초
78200개 이미지 생성 완료!! == 경과시간: 1144초
78300개 이미지 생성 

99500개 이미지 생성 완료!! == 경과시간: 1596초
99600개 이미지 생성 완료!! == 경과시간: 1598초
99700개 이미지 생성 완료!! == 경과시간: 1601초
99800개 이미지 생성 완료!! == 경과시간: 1603초
99900개 이미지 생성 완료!! == 경과시간: 1606초
100000개 이미지 생성 완료!! == 경과시간: 1608초
100100개 이미지 생성 완료!! == 경과시간: 1611초
100200개 이미지 생성 완료!! == 경과시간: 1613초
100300개 이미지 생성 완료!! == 경과시간: 1615초
100400개 이미지 생성 완료!! == 경과시간: 1617초
100500개 이미지 생성 완료!! == 경과시간: 1620초
100600개 이미지 생성 완료!! == 경과시간: 1622초
100700개 이미지 생성 완료!! == 경과시간: 1624초
100800개 이미지 생성 완료!! == 경과시간: 1626초
100900개 이미지 생성 완료!! == 경과시간: 1628초
101000개 이미지 생성 완료!! == 경과시간: 1630초
101100개 이미지 생성 완료!! == 경과시간: 1633초
101200개 이미지 생성 완료!! == 경과시간: 1635초
101300개 이미지 생성 완료!! == 경과시간: 1637초
101400개 이미지 생성 완료!! == 경과시간: 1639초
101500개 이미지 생성 완료!! == 경과시간: 1641초
101600개 이미지 생성 완료!! == 경과시간: 1643초
101700개 이미지 생성 완료!! == 경과시간: 1645초
101800개 이미지 생성 완료!! == 경과시간: 1647초
101900개 이미지 생성 완료!! == 경과시간: 1649초
102000개 이미지 생성 완료!! == 경과시간: 1652초
102100개 이미지 생성 완료!! == 경과시간: 1654초
102200개 이미지 생성 완료!! == 경과시간: 1656초
102300개 이미지 생성 완료!! == 경과

123000개 이미지 생성 완료!! == 경과시간: 2133초
123100개 이미지 생성 완료!! == 경과시간: 2135초
123200개 이미지 생성 완료!! == 경과시간: 2137초
123300개 이미지 생성 완료!! == 경과시간: 2140초
123400개 이미지 생성 완료!! == 경과시간: 2142초
123500개 이미지 생성 완료!! == 경과시간: 2145초
123600개 이미지 생성 완료!! == 경과시간: 2147초
123700개 이미지 생성 완료!! == 경과시간: 2149초
123800개 이미지 생성 완료!! == 경과시간: 2152초
123900개 이미지 생성 완료!! == 경과시간: 2154초
124000개 이미지 생성 완료!! == 경과시간: 2157초
124100개 이미지 생성 완료!! == 경과시간: 2160초
124200개 이미지 생성 완료!! == 경과시간: 2163초
124300개 이미지 생성 완료!! == 경과시간: 2165초
124400개 이미지 생성 완료!! == 경과시간: 2167초
124500개 이미지 생성 완료!! == 경과시간: 2170초
124600개 이미지 생성 완료!! == 경과시간: 2172초
124700개 이미지 생성 완료!! == 경과시간: 2176초
124800개 이미지 생성 완료!! == 경과시간: 2179초
124900개 이미지 생성 완료!! == 경과시간: 2182초
125000개 이미지 생성 완료!! == 경과시간: 2185초
125100개 이미지 생성 완료!! == 경과시간: 2188초
125200개 이미지 생성 완료!! == 경과시간: 2190초
125300개 이미지 생성 완료!! == 경과시간: 2193초
125400개 이미지 생성 완료!! == 경과시간: 2195초
125500개 이미지 생성 완료!! == 경과시간: 2197초
125600개 이미지 생성 완료!! == 경과시간: 2200초
125700개 이미지 생성 완료!! == 경과시간: 2202초
125800개 이미지 생성 완료!! 

146500개 이미지 생성 완료!! == 경과시간: 2760초
146600개 이미지 생성 완료!! == 경과시간: 2763초
146700개 이미지 생성 완료!! == 경과시간: 2766초
146800개 이미지 생성 완료!! == 경과시간: 2769초
146900개 이미지 생성 완료!! == 경과시간: 2772초
147000개 이미지 생성 완료!! == 경과시간: 2775초
147100개 이미지 생성 완료!! == 경과시간: 2778초
147200개 이미지 생성 완료!! == 경과시간: 2782초
147300개 이미지 생성 완료!! == 경과시간: 2785초
147400개 이미지 생성 완료!! == 경과시간: 2788초
147500개 이미지 생성 완료!! == 경과시간: 2791초
147600개 이미지 생성 완료!! == 경과시간: 2794초
147700개 이미지 생성 완료!! == 경과시간: 2797초
147800개 이미지 생성 완료!! == 경과시간: 2799초
147900개 이미지 생성 완료!! == 경과시간: 2802초
148000개 이미지 생성 완료!! == 경과시간: 2805초
148100개 이미지 생성 완료!! == 경과시간: 2808초
148200개 이미지 생성 완료!! == 경과시간: 2811초
148300개 이미지 생성 완료!! == 경과시간: 2814초
148400개 이미지 생성 완료!! == 경과시간: 2817초
148500개 이미지 생성 완료!! == 경과시간: 2820초
148600개 이미지 생성 완료!! == 경과시간: 2823초
148700개 이미지 생성 완료!! == 경과시간: 2826초
148800개 이미지 생성 완료!! == 경과시간: 2829초
148900개 이미지 생성 완료!! == 경과시간: 2832초
149000개 이미지 생성 완료!! == 경과시간: 2835초
149100개 이미지 생성 완료!! == 경과시간: 2839초
149200개 이미지 생성 완료!! == 경과시간: 2842초
149300개 이미지 생성 완료!! 

In [128]:
textimagegenerator(NUM_IMAGES = 20000,
                   save_image_path = 'data/test',
                   save_label_path = 'data/test_label')

100개 이미지 생성 완료!! == 경과시간: 1초
200개 이미지 생성 완료!! == 경과시간: 2초
300개 이미지 생성 완료!! == 경과시간: 2초
400개 이미지 생성 완료!! == 경과시간: 3초
500개 이미지 생성 완료!! == 경과시간: 4초
600개 이미지 생성 완료!! == 경과시간: 5초
700개 이미지 생성 완료!! == 경과시간: 6초
800개 이미지 생성 완료!! == 경과시간: 8초
900개 이미지 생성 완료!! == 경과시간: 10초
1000개 이미지 생성 완료!! == 경과시간: 12초
1100개 이미지 생성 완료!! == 경과시간: 13초
1200개 이미지 생성 완료!! == 경과시간: 15초
1300개 이미지 생성 완료!! == 경과시간: 16초
1400개 이미지 생성 완료!! == 경과시간: 17초
1500개 이미지 생성 완료!! == 경과시간: 19초
1600개 이미지 생성 완료!! == 경과시간: 20초
1700개 이미지 생성 완료!! == 경과시간: 21초
1800개 이미지 생성 완료!! == 경과시간: 22초
1900개 이미지 생성 완료!! == 경과시간: 23초
2000개 이미지 생성 완료!! == 경과시간: 25초
2100개 이미지 생성 완료!! == 경과시간: 26초
2200개 이미지 생성 완료!! == 경과시간: 27초
2300개 이미지 생성 완료!! == 경과시간: 29초
2400개 이미지 생성 완료!! == 경과시간: 30초
2500개 이미지 생성 완료!! == 경과시간: 31초
2600개 이미지 생성 완료!! == 경과시간: 33초
2700개 이미지 생성 완료!! == 경과시간: 34초
2800개 이미지 생성 완료!! == 경과시간: 35초
2900개 이미지 생성 완료!! == 경과시간: 36초
3000개 이미지 생성 완료!! == 경과시간: 37초
3100개 이미지 생성 완료!! == 경과시간: 38초
3200개 이미지 생성 완료!! == 경과시간: 40초
3300개 이미지 생성 완료!! == 경과시간